In [1]:
%load_ext autoreload

In [2]:
import sys
import os
import json
import pandas as pd

os.environ['CUDA_VISIBLE_DEVICES']='1'
pd.set_option('display.max_colwidth', None)

In [18]:
def load_data(data_path):
    data = []
    with open(data_path) as file:
        for line in file.readlines()[:1000]:
            obj = json.loads(line)
            data.append({
                'id' : obj['id'],
                'op' : obj['op'],
                'source_keyphrases': [x['keyphrases'] for x in obj['target_retrieved_passages']],
                'target_keyphrases': [x['selected_keyphrases'] for x in obj['target_counterarg']],
                'target_sents': [x['tokens'] for x in obj['target_counterarg']],
            })
            
    return data

In [19]:
data = load_data('../../../data-ceph/arguana/arg-generation/argument-undermining/data/candela/data/arggen/train.jsonl')

In [20]:
df = pd.DataFrame(data)

In [21]:
df.sample(10).head(n=10)

,id,op,source_keyphrases,target_keyphrases,target_sents
511,1b3xqg,"[i, do, n't, understand, gay, people, ., (, x-posted, to, /, offmychest, ), okay, ., gays, ., let, 's, talk, about, them, for, a, second, (, because, they, 're, just, not, getting, enough, media, coverage, these, days, ), ., because, seriously, ,, i, do, n't, understand, their, mentalities, at, all, ., or, their, reasonings, ., this, may, come, off, a, little, rough, ,, but, i, am, seriously, just, trying, to, get, this, off, my, chest, because, it, has, been, bothering, me, a, lot, lately, ., first, :, i, grew, up, in, a, home, where, man, +, woman, =, good, and, ...]","[[gay people, exposure, undersecretary, mccarthy, roy cohn, regarded homosexuality, would blackmail gay employees, classified information, allowed 91 gays, was a closeted gay man, investigated homosexuality], [many private companies, washington, workers, johnson, the number, home, gays, today, fear, subculture, being homosexual, naming other gay men], [hundreds, gay.the executive order, paper, effect, the order, policy, admittance, gays, signed executive order 10450, set security standards, set in place, put in place], [callers, threat, islamic state, shame, order, public morality, terrorism, calling homosexuality, would stand against gays], [reuters, uganda, channels, the minister, organizations, the list, promotion, parliament, calls, life imprisonment, the proposal, aid, receiving support, accused gays, providing financial support, drop this clause]]","[[set security standards, the list, islamic state], [gay people, hundreds]]","[[in, the, 1950s, ,, homosexuals, were, on, the, same, state, department, list, as, anarchists, and, communists, as, threats, to, national, security, .], [hundreds, of, people, were, fired, from, their, government, jobs, for, being, homosexuals, ,, thousands, of, people, were, kicked, out, of, the, military, ,, and, thousands, more, were, denied, jobs, in, government, for, the, same, reason, .]]"
430,1b5nhm,"[i, think, the, death, penalty, is, more, rational, than, life, in, prison, cmv, i, believe, that, executing, a, violent, offender, (, i, 'm, defining, this, as, a, murderer, or, a, rapist, ), is, more, rational, than, life, in, prison, ., i, do, n't, believe, murderers, and, rapists, can, be, successfully, reintregated, into, society, and, because, of, this, they, provide, no, value, to, society, ., thus, ,, it, makes, more, sense, to, kill, a, violent, offender, than, to, waste, a, bunch, of, money, on, keeping, him, alive, in, prison, ., i, do, n't, see, how, that, benefits, society, ., i, realize, ...]","[[harm, rights, support the death penalty, having the state, execute citizens, were guilty], [the victims, serial killers, fyi, no innocent person, innocence, the death penalty, would qualify for capital punishment], [june, hedge fund, clients, federal, a prison term, the financial loss, the public, convictions, life sentences, murderers, the lawyer, recent years, enron, securities fraud, sentencing guidelines, serving a 24-year sentence], [excuse, amusement, uva murderer, sons, duke, line, the school, proven innocent, is a fine school]]","[[murderers], [no innocent person, proven innocent, execute citizens], [a prison term, life sentences, no innocent person, proven innocent]]","[[there, are, murderers, and, rapists, that, are, wrongly, accused, .], [so, if, we, execute, them, and, later, find, out, they, are, innocent, ,, there, is, nothing, we, can, do, about, it, .], [so, if, we, keep, them, in, prison, for, life, ,, we, are, always, able, to, free, them, if, they, are, found, out, to, be, innocent]]"
94,1bd4tu,"[i, do, n't, think, that, people, of, low, intelligence, should, have, every, right, that, an, intelligent, person, has, ., i, do, n't, mean, people, with, disabilities, ., i, 'm, talking, about, people, that, can, hardly, understand, words, like, ..., idk, ..., acronym, or, something, ., i, do, n't, think, it, 's, fair, ,, that, in, our, society